In [1]:
using DrWatson
@quickactivate :Catalyst

┌ Info: Precompiling Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
└ @ Base loading.jl:1260


In [2]:
using Distributed
N = 12
addprocs(N*3)

36-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 26
 27
 28
 29
 30
 31
 32
 33
 34
 35
 36
 37

In [3]:
nprocs()

37

In [5]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/amjalled/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [6]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst
using CSV

      From worker 21:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 36:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 23:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 8:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 22:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 14:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 17:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 7:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 3:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 4:	 Activating environment at `~/Repos/advection-diffusion-catalysi

In [7]:
@everywhere function eval_model(input_args::Array,w::Float64, input_exp::Array, output_exp::Array, micromesh) 
    try 
        return Catalyst.solve(input_args[1], input_args[2], input_args[3], 
                    input_exp, output_exp, progress=false, 
                    microcomp_type=:nonlinear, 
                    Q=input_args[4], kₙ=input_args[5], 
                    calibration=true, w=w, micromesh=micromesh)
    catch err #in case we loose newton convergence in a microscopic computation 
        return 1e10
    end
end

function calibrate(cma,N)
    w = [1.5302e-4*(1/0.37), 1.9128e-4*(1/0.37), 2.2954e-4*(1/0.37)] # Q8, Q10, Q12
    Init = [0.00490394, 5.3789, 1.07486, 21.2586, 0.841337] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0, 0], [1, 50, 50, 30, 30]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.1, opts) 
    
    #Step 1
    #Read in all 3 different velocity curves
    input_exp_12 = []
    output_exp_12 = []
    input_exp_10 = []
    output_exp_10 = []
    input_exp_8 = []
    output_exp_8 = []
    
    for row in CSV.File(datadir("experiment/SBA15_interp_12.csv"); delim = ",")
        push!(input_exp_12, row.I)
        push!(output_exp_12, row.O)
    end
    
    for row in CSV.File(datadir("experiment/SBA15_interp_10.csv"); delim = ",")
        push!(input_exp_10, row.I)
        push!(output_exp_10, row.O)
    end
    
    for row in CSV.File(datadir("experiment/SBA15_interp_8.csv"); delim = ",")
        push!(input_exp_8, row.I)
        push!(output_exp_8, row.O)
    end
    
    n = 2
    nels = (5n, convert(Int,2.5n),n)
    S = Vec((0.0,0.0,0.0))
    E = Vec((1.0,0.5,0.2))
    grid = generate_grid(Tetrahedron, nels, S, E)
    faces = union(getfaceset(grid,"top"),getfaceset(grid,"bottom"),getfaceset(grid,"left"),getfaceset(grid,"right"),
                   getfaceset(grid,"front"),getfaceset(grid,"back"))
    addfaceset!(grid, "1", faces)

    #Step 2 change the model evaluation
    while isempty(es.stop()) 
        solutions = es.ask() # Returns array of array where inner array holds set of parameters length outer array = N
        #Check the syntax here
        F = Array{Future}(undef, N*3)
        errors = zeros(N*3)
        
        #Complete the for loop
        s = 1
        for idx in 1:length(errors)
            if idx % 3 == 1
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[3], input_exp_12, output_exp_12, grid))
            elseif idx % 3 == 2
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[2], input_exp_10, output_exp_10,grid))
            elseif idx % 3 == 0
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[1], input_exp_8, output_exp_8,grid))
                s += 1
            end
        end
       
        
        # fetch the errors entry 
        for i in 1:N*3
            errors[i] = fetch(F[i])
        end
                
        # Reduce the N*3 length array errors to an array of length N (adding up errors)
        errors_red = sum.(Iterators.partition(errors,3))
                 
        es.tell(solutions,errors_red) 
        #This is only for saving/logging/printing current status
        es.logger.add() 
        es.disp() 
    end
return errors_red
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 5 (seed=1007447, Wed Aug 19 13:08:22 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 1.968817117849173e+01 1.0e+00 9.19e-02  8e-02  1e-01 27:46.0
    2     24 2.115253216677636e+01 1.3e+00 8.65e-02  7e-02  9e-02 52:43.9
    3     36 2.186252220613483e+01 1.6e+00 8.19e-02  6e-02  9e-02 78:15.3
    4     48 2.566828485037656e+01 1.8e+00 6.77e-02  4e-02  7e-02 103:36.4
    5     60 2.288920220039327e+01 1.8e+00 6.44e-02  4e-02  7e-02 128:28.8
    6     72 1.700483663022409e+01 2.4e+00 6.07e-02  4e-02  6e-02 153:23.4
    7     84 1.848706092815302e+01 2.6e+00 5.89e-02  3e-02  6e-02 177:03.1
    8     96 1.722262602001227e+01 2.7e+00 6.83e-02  4e-02  7e-02 203:43.8
    9    108 1.627618059549980e+01 2.4e+00 5.95e-02  3e-02  6e-02 229:18.1
   10    120 2.044341617926123e+01 2.3e+00 5.49e-02  3e-02  5e-02 254:40.0
   11    132 1.615517094090912e+01 2.3e+00 4.86e-02  3e-02  5e-02 279:31.1
   12    144 1